# Coursework 3

Problems are set by Dr. Wei Dai, 2022

Check the lecture notes for the steps of ADMM and the stopping criteria. 

In [30]:
using DataFrames, FFTW, Wavelets, Images, LinearAlgebra, Random, StatsBase

# Make sure that you complete the involvement table. 
# The first row is for CID number. 
# Other rows are for the involvement for each "big" problem (8 big problems in coursework 1). 
# "1" for getting involved in this part. 
# "0" for not involved
Contributions = DataFrame( A=[1234,0,0,0,0], B = [1234,0,0,0,0] )

,A,B
,Int64,Int64
1,1234,1234
2,0,0
3,0,0
4,0,0
5,0,0


## 1. Netflix Problem (40%)

The $\texttt{ratings.csv}$ is MovieLens data downloaded from [GroupLens](grouplens.org).

### 1.1 Prepare the data (6%)

1. Use the package $\texttt{xlsx.jl}$ or $\texttt{csv.jl}$ to read $\texttt{ratings.csv}$. Using the existing movie rating data to form a matrix $\bm{X}_0$, where the rows correspond to users and the columns correspond to movies. Note that for our purposes, we delete all the movies that have less than 15 ratings.$\newline$ Show the size of your matrix $\bm{X}_0$. Denote its size by $M \times N$.        

2. Based on your obtained $\bm{X}_0$, define 
        $$\begin{align*}
            \Delta & := \left\{(m,n):~ \text{a rating exists for the $m$-th user and $n$-th movie}  \right\}. 
        \end{align*}$$
        In Julia, we use $\texttt{Delta = Array{Int64,2}(undef,0,2)}$ to initialize $\Delta$. $\newline$ Find $\Delta$ ($\texttt{Delta}$ in Julia). Show the size of $\texttt{Delta}$. 
        
3. Use $\texttt{Random.seed!(1234)}$ to initialize the seed for random number generator. Use $\texttt{StatsBase.sample}$ to sample (without replacement) 80\% of the entries in $\texttt{Delta}$ to form $\texttt{Omega}$ ($\Omega$). (Use $\texttt{ceil}$ function to calculate the size of $\Omega$.)
$\newline$
Write a function $\texttt{pOmega}$ to obtain $\bm{y} = \mathcal{P}_{\Omega}(\bm{X}_0)$. 
$\newline$
The vector $\bm{y}$ contains our training data. The vector $\bm{y}_e = \mathcal{P}_{\Delta \backslash \Omega}(\bm{X}_0)$ gives the test data for performance evaluation. 
$\newline$
Find both $\bm{y}$ and $\bm{y}_e$. 


### 1.2 Employ alternating optimization algorithm (10%)

In this part, we are solving $$\begin{align*}
        & \min_{\bm{X}}~ \delta\left( \text{rank}(\bm{X}) \le 10 \right)
        + \frac{1}{2} \lVert \bm{y} - \mathcal{P}_{\Omega}(\bm{X}) \rVert_2^2
    \end{align*}$$
using alternating minimization. More specifically, let $\bm{L} \in \mathbb{R}^{M \times 10}$ and $\bm{R} \in \mathbb{R}^{N \times 10}$. We solve 
$$\begin{align*}
    & \min_{\bm{L},\bm{R}}~ \lVert \bm{y} - \mathcal{P}_{\Omega}(\bm{L} \bm{R}^{\mathsf{T}}) \rVert_2^2
\end{align*}$$
in an alternating way.

1. Implement the alternating minimization algorithm. Test it with randomly initialized $\bm{L}$ and $\bm{R}$ using $\texttt{randn(Float64,M,10)}$ and $\texttt{randn(Float64,N,10)}$ respectively. Denote the output of your alternating minimization algorithm $\hat{\bm{X}}$. 

2. Find the training performance $\lVert \mathcal{P}_{\Omega} (\hat{\bm{X}} - \bm{X}_0)\rVert_2^2 / |\Omega|$ and the prediction performance $\lVert \mathcal{P}_{\Delta \backslash \Omega} (\hat{\bm{X}} - \bm{X}_0)\rVert_2^2 / |\Delta \backslash \Omega|$. 

### 1.3 Employ proximal gradient method (10%)

In this part, we are solving $$\begin{align*}
        & \min_{\bm{X}}~ \lambda \lVert \bm{X} \rVert_*
        + \frac{1}{2} \lVert \bm{y} - \mathcal{P}_{\Omega}(\bm{X}) \rVert_2^2
    \end{align*}$$
    where $\lambda>0$ is chosen so that your result $\hat{\bm{X}}$ has a rank close to 10 (all the singular values except the first 10 ones are close to 10). More specifically, use the proximal gradient method to solve the above convex optimization problem. 

1. Give the closed form of the gradient of the term $\frac{1}{2} \lVert \bm{y} - \mathcal{P}_{\Omega}(\bm{X}) \rVert_2^2$ in terms of $\mathcal{P}_{\Omega}$ and $\bm{X}$.
        
2. Implement the proximal gradient method. 
        
3. Find an appropriate value for $\lambda >0$. Test the training performance and the prediction performance.

### 1.4 Employ ADMM (14%)

A more practical approach is to solve the following constrained optimization problem 
    $$\begin{align*}
        \min_{\bm{X}}~
        & \lambda \lVert \bm{X} \rVert_*
        + \frac{1}{2} \lVert \bm{y} - \mathcal{P}_{\Omega}(\bm{X}) \rVert_2^2 \\
        \text{s.t.}~
        & 0 \le X_{m,n} \le 5,~ \forall~1 \le m \le M,~ 1 \le n \le N. 
    \end{align*}$$
We simplify the notation by writing $0 \le \bm{X} \le 5$ for the constraint.

We use ADMM to solve the constrained optimization problem. An ADMM Lagrangian is given by 
$$\begin{align*}
    \min_{\bm{X},\bm{Z}}~ 
    & \frac{1}{2} \lVert \bm{y} - \mathcal{P}_{\Omega}(\bm{X}) \rVert_2^2 
    + \lambda \lVert \bm{Z}_1 \rVert_*
    + \delta \left( 0 \le \bm{Z}_2 \le 5 \right) \\
    & + \left\langle \bm{\Lambda}_1,\bm{X}-\bm{Z}_1 \right\rangle
    + \left\langle \bm{\Lambda}_2,\bm{X}-\bm{Z}_2 \right\rangle \\
    & + \frac{\rho}{2} \lVert \bm{X}-\bm{Z}_1 \rVert_F^2 
    + \frac{\rho}{2} \lVert \bm{X}-\bm{Z}_2 \rVert_F^2. 
\end{align*}$$

1. Find the closed form for minimization with respect to $\bm{X}$ for given $\bm{Z}$ and $\bm{\Lambda}$.

2. Find the closed form for minimization with respect to $\bm{Z}$ for given $\bm{X}$ and $\bm{\Lambda}$.         

3. Implement the ADMM procedure. 

4. With properly chosen $\lambda > 0$, test the training performance and the prediction performance. 

## 2. Video Background and Foreground Separation (20%)

In this application, we use the video file $\texttt{demo.avi}$.

### 2.1 Data preparation (5%)

The goal of this step is to form a matrix where each column is a vectorized version of a frame of the video and the number of columns is the number of the frames in the video. Denote the matrix by $\bm{Y}$.

With help from GTA Mr. Zhengang Guo, we have identified that the function $\texttt{VideoIO.load}$ can load the video file and that the function $\texttt{VideoIO.channelview}$ can get frames from the loaded video. 

Write a function $\texttt{video2matrix}$ to form the video matrix $\bm{Y}$ from the video file. 


### 2.2 Employ ADMM (15%)

The video matrix $\bm{Y}$ can be modelled as 
    $$\begin{align*}
        & \bm{Y} = \bm{L} + \bm{S} + \bm{W},
    \end{align*}$$
where $\bm{L}$ is a low rank matrix to represent the background, $\bm{S}$ is a sparse matrix (only a few entries are nonzero) that correspond to moving foreground, and $\bm{W}$ denotes Gaussian noise.

We consider the following constrained optimization problem 
$$\begin{align*}
    \min_{\bm{L},\bm{S}}~ 
    & \frac{1}{2} \lVert \bm{Y} - \bm{L} -\bm{S} \rVert_F^2
    + \lambda_1 \lVert \bm{L} \rVert_* 
    + \lambda_2 \lVert \bm{S} \rVert_1 \\
    \text{s.t.}~
    & 0 \le \bm{L} \le 255,  
\end{align*}$$
where with slight abuse of notation $\lVert \bm{S} \rVert_1 := \lVert \text{vect}(\bm{S}) \rVert_1$. 
    An ADMM optimization formulation is given below. 
$$\begin{align*}
    \min_{\bm{L},\bm{S}, \bm{Z}}~ 
    & \frac{1}{2} \lVert \bm{Y} - \bm{L} -\bm{S} \rVert_F^2
    + \lambda_1 \lVert \bm{Z}_1 \rVert_* 
    + \delta \left( 0 \le \bm{Z}_2 \le 255 \right)
    + \lambda_2 \lVert \bm{Z}_3 \rVert_1 \\
    \text{s.t.}~
    & \bm{L} = \bm{Z}_1,~
    \bm{L} = \bm{Z}_2,~
    \bm{S} = \bm{Z}_3. 
\end{align*}$$

1. Find the closed form for minimization with respect to $(\bm{L},\bm{S})$ for given $\bm{Z}$ and $\Lambda$. 

2. Find the closed form for minimization with respect to $\bm{Z}$ for given $(\bm{L},\bm{S})$ and $\Lambda$. 

3. Implement the corresponding ADMM procedure. 

4. Choose reasonable parameters $\lambda_1$ and $\lambda_2$ to have nice results. Save the estimated $\bm{L}$ and $\bm{S}$ into video files $\texttt{Background.avi}$ and $\texttt{Foreground.avi}$ respectively. (Upload these files when you submit your coursework.)

## 3. MRI Compressed Sensing Recovery (40%)

The following optimization problem can be formulated for MRI compressed sensing recovery. 
$$\begin{align*}
    \min_{\bm{X}}~
    & \frac{1}{2} \lVert \bm{y} - \mathcal{P}_{\Omega}\left( \mathcal{F} \left( \bm{X} \right) \right) \rVert_2^2
    + \alpha \lVert \mathcal{H}(\bm{X}) \rVert_1\\
    & + \beta \lVert \mathcal{D}_1(\bm{X}) \rVert_1
    + \beta \lVert \mathcal{D}_2(\bm{X}) \rVert_1
    + \beta \lVert \mathcal{D}_3(\bm{X}) \rVert_1
    + \beta \lVert \mathcal{D}_4(\bm{X}) \rVert_1 \\
    \text{s.t.}~
    & 0 \le \bm{X} \le 255,
\end{align*}$$
where $\mathcal{H}$ stands for the discrete Haar wavelet transform (See the section Haar matrix at [Wikipedia](https://en.wikipedia.org/wiki/Haar_wavelet). $\mathcal{D}_1(\bm{X})$, $\mathcal{D}_2(\bm{X})$, $\mathcal{D}_3(\bm{X})$, and $\mathcal{D}_4(\bm{X})$ compute $X_{m,n} - X_{m-1,n}$, $X_{m,n} - X_{m,n-1}$, $X_{m,n} - X_{m-1,n-1}$, $X_{m,n} - X_{m-1,n+1}$, respectively, for all valid $m,n$ to form the corresponding vetors.


### 3.1 (5%)
Find the form of the adjoint operators for $\mathcal{P}_{\Omega}(\mathcal{F}(\cdot))$, $\mathcal{H}$, $\mathcal{D}_1$, $\mathcal{D}_2$, $\mathcal{D}_3$, and $\mathcal{D}_4$.    
    

We have 
$$ min_X \frac{1}{2}\|y-P_\Omega(F(\bm{X}))\|^2_2 + \alpha\|H(\bm{X})\|_1 + \beta\|D_1(\bm{X})\|_1 + \beta\|D_2(\bm{X})\|_1 + \beta\|D_3(\bm{X})\|_1 + \beta\|D_4(\bm{X})\|_1 $$
s.t.
$$ 0 \leq \bm{X} \leq 255 $$

The adjoint is defined s.t. 
$$ <Ax,y> = <x,A^*y> $$
i.e. conjugate-transpose matrix multiply

Therefore:

Adjoint of $P_\Omega(F(.))$:
$$ (P_\Omega(F(.)))^* = F^*(P^*_\Omega(.)) $$
It is worth noting that for the Fourier Transform, $F^* = F^{-1}$, i.e. the adjoint is equal to the inverse

Adjoint of $H(X)$:
$$ (H(X))^* = X^*H^* $$

Adjoint of $D_i(X)$:
$$ (D_i(X))^* = X^*D_i^* $$
for i = 1...4

### 3.2 (5%)

Find an ADMM optimization formulation so that each ADMM step can be solved using proximal operators with closed form solutions. 

ADMM optimization formulation:
$$ min \frac{1}{2}\|y-P_\Omega(F(\bm{X}))\|^2_2 + \alpha\|Z_0\|_1+\beta\|Z_1\|_1+\beta\|Z_2\|_1+\beta\|Z_3\|_1+\beta\|Z_4\|_1 + \delta_{\geq 0}(Z_5) + \delta_{\leq 255}(Z_6)$$
s.t.
$$ H(X) = Z_0 $$
$$ D_i(X) = Z_i $$
$$ X + Z_5 = 0 $$
$$ X + Z_6 = 255 $$

### 3.3 (5%)

Find the corresponding ADMM Lagrangian. Note that complex numbers are involved in the formulation due to the Fourier transform. The inner products of Lagrange multipliers and variables are in the form of $\Re \left(\langle \bm{\lambda},\bm{x} \rangle\right) $ where $\Re$ denotes the real part of the number.

Augmented Lagrangian:
$$ L_p(X, Z_{0...6}, v)  = min \frac{1}{2}\|y-P_\Omega(F(\bm{X}))\|^2_2 + \alpha\|Z_0\|_1+\beta\|Z_1\|_1+\beta\|Z_2\|_1+\beta\|Z_3\|_1+\beta\|Z_4\|_1 + \delta_{\geq 0}(Z_5) + \delta_{\leq 255}(Z_6) + v^T(
   \begin{bmatrix}
    H \\
    D_1 \\
    D_2 \\
    D_3 \\
    D_4 \\
    I \\
    I \\
\end{bmatrix}
    X
+
\begin{bmatrix}
    -I \\
    -I \\
    -I \\
    -I \\
    -I \\
    I \\
    I \\
\end{bmatrix}
\begin{bmatrix}
    Z_0 \\
    Z_1 \\
    Z_2 \\
    Z_3 \\
    Z_4 \\
    Z_5 \\
    Z_6 \\
\end{bmatrix}
- 
\\
\begin{bmatrix}
    0 \\
    0 \\
    0 \\
    0 \\
    0 \\
    0 \\
    255 \\
\end{bmatrix})
+ \frac{\rho}{2}\|
\begin{bmatrix}
    H \\
    D_1 \\
    D_2 \\
    D_3 \\
    D_4 \\
    I \\
    I \\
\end{bmatrix}
    X
+
\\
\begin{bmatrix}
    -I \\
    -I \\
    -I \\
    -I \\
    -I \\
    I \\
    I \\
\end{bmatrix}
\begin{bmatrix}
    Z_0 \\
    Z_1 \\
    Z_2 \\
    Z_3 \\
    Z_4 \\
    Z_5 \\
    Z_6 \\
\end{bmatrix}
- 
\begin{bmatrix}
    0 \\
    0 \\
    0 \\
    0 \\
    0 \\
    0 \\
    255 \\
\end{bmatrix}\|^2_2$$

### 3.4 (5%)

Find the closed form solutions for the ADMM steps. Write your solutions using the operators and the adjoint operators.

$$ X^{k+1} = min_x \frac{1}{2}\|y-P_\Omega(F(X))\|^2_2 + \frac{\rho}{2}\|
\begin{bmatrix}
    H \\
    D_1 \\
    D_2 \\
    D_3 \\
    D_4 \\
    I \\
    I \\
\end{bmatrix}^*
    X
+
\begin{bmatrix}
    -I \\
    -I \\
    -I \\
    -I \\
    -I \\
    I \\
    I \\
\end{bmatrix}^*
\begin{bmatrix}
    Z_0 \\
    Z_1 \\
    Z_2 \\
    Z_3 \\
    Z_4 \\
    Z_5 \\
    Z_6 \\
\end{bmatrix}
- 
\begin{bmatrix}
    0 \\
    0 \\
    0 \\
    0 \\
    0 \\
    0 \\
    255 \\
\end{bmatrix}
+
\frac{v^k}{\rho}\|^2_2$$

$$ Z^{k+1} = min_{Z_0...Z_6} \alpha \|Z_0\|_1 + \frac{\rho}{2}\|H(X^{k+1})-Z_0+\frac{v_0^k}{\rho}\|^2_2$$
$$ + \beta \|Z_1\|_1 + \frac{\rho}{2}\|D_1(X^{k+1})-Z_1+\frac{v_1^k}{\rho}\|^2_2 $$
$$ + \beta \|Z_2\|_1 + \frac{\rho}{2}\|D_2(X^{k+1})-Z_2+\frac{v_2^k}{\rho}\|^2_2 $$
$$ + \beta \|Z_3\|_1 + \frac{\rho}{2}\|D_3(X^{k+1})-Z_3+\frac{v_3^k}{\rho}\|^2_2 $$
$$ + \beta \|Z_4\|_1 + \frac{\rho}{2}\|D_4(X^{k+1})-Z_4+\frac{v_4^k}{\rho}\|^2_2 $$
$$ + \delta_{\geq 0}(Z_5) + \frac{\rho}{2}\|X^{k+1}+Z_5+\frac{v_5^k}{\rho}\|^2_2 $$
$$ + \delta_{\leq 255}(Z_6) + \frac{\rho}{2}\|X^{k+1}+Z_6-255\bm{I}+\frac{v_6^k}{\rho}\|^2_2 $$

$$ V^{k+1} = V^k + \rho(
    \begin{bmatrix}
    H \\
    D_1 \\
    D_2 \\
    D_3 \\
    D_4 \\
    I \\
    I \\
\end{bmatrix}^*
X^{k+1} + 
\begin{bmatrix}
    -I \\
    -I \\
    -I \\
    -I \\
    -I \\
    I \\
    I \\
\end{bmatrix}^*
- 
\begin{bmatrix}
    0 \\
    255 \\
\end{bmatrix}^*
)$$

For $X^{k+1}$: This is essentially a Least Squares Problem:
$$  F^*P^*_\Omega\|y-P_\Omega F(X)\|_2 + \rho 
\begin{bmatrix}
    H \\
    D_1 \\
    D_2 \\
    D_3 \\
    D_4 \\
    I \\
    I \\
\end{bmatrix}
\|
\begin{bmatrix}
    H \\
    D_1 \\
    D_2 \\
    D_3 \\
    D_4 \\
    I \\
    I \\
\end{bmatrix}^*
    X
+
\begin{bmatrix}
    -I \\
    -I \\
    -I \\
    -I \\
    -I \\
    I \\
    I \\
\end{bmatrix}^*
\begin{bmatrix}
    Z_0 \\
    Z_1 \\
    Z_2 \\
    Z_3 \\
    Z_4 \\
    Z_5 \\
    Z_6 \\
\end{bmatrix}
- 
\begin{bmatrix}
    0 \\
    0 \\
    0 \\
    0 \\
    0 \\
    0 \\
    255 \\
\end{bmatrix}
\|_2 = 0
$$

$$  F^*P^*_\Omega y- F^*P^*_\Omega P_\Omega F(X) + \rho 
\begin{bmatrix}
    H \\
    D_1 \\
    D_2 \\
    D_3 \\
    D_4 \\
    I \\
    I \\
\end{bmatrix}
\begin{bmatrix}
    H \\
    D_1 \\
    D_2 \\
    D_3 \\
    D_4 \\
    I \\
    I \\
\end{bmatrix}^*
    X
+
\rho
\begin{bmatrix}
    -I \\
    -I \\
    -I \\
    -I \\
    -I \\
    I \\
    I \\
\end{bmatrix}^*
\begin{bmatrix}
    Z_0 \\
    Z_1 \\
    Z_2 \\
    Z_3 \\
    Z_4 \\
    Z_5 \\
    Z_6 \\
\end{bmatrix}
- 
\rho
\begin{bmatrix}
    0 \\
    0 \\
    0 \\
    0 \\
    0 \\
    0 \\
    255 \\
\end{bmatrix}
= 0
$$

$$
F^*P^*_\Omega y + \rho 
\begin{bmatrix}
    -I \\
    -I \\
    -I \\
    -I \\
    -I \\
    I \\
    I \\
\end{bmatrix}^*
\begin{bmatrix}
    Z_0 \\
    Z_1 \\
    Z_2 \\
    Z_3 \\
    Z_4 \\
    Z_5 \\
    Z_6 \\
\end{bmatrix}
- 
\rho
\begin{bmatrix}
    0 \\
    0 \\
    0 \\
    0 \\
    0 \\
    0 \\
    255 \\
\end{bmatrix}
= F^*P^*_\Omega P_\Omega FX^{k+1} 
-
\begin{bmatrix}
    H \\
    D_1 \\
    D_2 \\
    D_3 \\
    D_4 \\
    I \\
    I \\
\end{bmatrix}
\begin{bmatrix}
    H \\
    D_1 \\
    D_2 \\
    D_3 \\
    D_4 \\
    I \\
    I \\
\end{bmatrix}^*
    X^{k+1}
$$

Therefore:

$$
X^{k+1} = 
(F^*P^*_\Omega P_\Omega F - 
\begin{bmatrix}
    H \\
    D_1 \\
    D_2 \\
    D_3 \\
    D_4 \\
    I \\
    I \\
\end{bmatrix}
\begin{bmatrix}
    H \\
    D_1 \\
    D_2 \\
    D_3 \\
    D_4 \\
    I \\
    I \\
\end{bmatrix}^*)^{-1}
(
F^*P^*_\Omega y + \rho 
\begin{bmatrix}
    -I \\
    -I \\
    -I \\
    -I \\
    -I \\
    I \\
    I \\
\end{bmatrix}^*
\begin{bmatrix}
    Z_0 \\
    Z_1 \\
    Z_2 \\
    Z_3 \\
    Z_4 \\
    Z_5 \\
    Z_6 \\
\end{bmatrix}
- 
\rho
\begin{bmatrix}
    0 \\
    0 \\
    0 \\
    0 \\
    0 \\
    0 \\
    255 \\
\end{bmatrix}
)
$$


For $Z_0^{k+1}$:
$$ Z_0^{k+1} = \mu_i (H(X^{k+1})+V_0^k) $$

For $Z_i^{k+1}$, i = 1...4:
$$ Z_i^{k+1} = \mu_i (D_i(X^{k+1})+V_i^{k}) $$

For $Z_5^{k+1}$:
$$ Z_5^{k+1} = Proj_{C5} (X^{k+1}+V_5^{k}) $$

For $Z_6^{k+1}$:
$$ Z_6^{k+1} = Proj_{C6} (X^{k+1}+V_6^{k}) $$

Where $\mu_i$ is the soft thresholding function and $Proj_C$ is the projection operator onto the C subspace 

### 3.5 (20%)

Implement an ADMM and use the data in the previous coursework to test the performance. 

1. Write functions (or use existing functions in Julia) for the involved operators $\mathcal{P}_{\Omega}(\mathcal{F}(\cdot))$, $\mathcal{H}$, $\mathcal{D}_1$, $\mathcal{D}_2$, $\mathcal{D}_3$, $\mathcal{D}_4$, and their adjoint operators. Note that the explicit matrix forms will lead to large storage space and slow computation speed. We expect that you do not use the explicit matrix forms. (5%)

2. Write functions to solve the involved proximal operators and least squares problems. As the involved linear operators are implemented using functions (not matrices), the conjugate gradient (CG) method are needed. The description of the CG method can be found from Wikipedia. A Julia implementation of can be found from the package $\texttt{IterativeSolvers.jl}$. You will also need $\texttt{LinearMaps.jl}$ as our linear operators are implemented as functions rather than matrices. (10%)

3. ADMM implementation and test. (5%)

In [31]:
## Projection & Fourier Transform
function proj_ft(X)
    return fft(X)
end

## Haar wavelet
harr_filt = wavelet(WT.haar, WT.Filter)

OrthoFilter{Wavelets.WT.PerBoundary}([0.7071067811865476, 0.7071067811865476], "haar")

In [62]:
## Difference functions
function d1(X)
    X_shape = size(X)
    D_1 = Array(zeros(size(X)))

    for i in 2:X_shape[1]
        print(X[i,:], " .- ", X[i-1,:], " .= ", X[i,:] .- X[i-1,:], "\n")
        D_1[i,:] .= X[i,:] .- X[i-1,:]
    end

    return D_1
end

function d2(X)
    X_shape = size(X)
    D_2 = Array(zeros(size(X)))

    for i in 2:X_shape[2]
        print(X[:,i], " .- ", X[:,i-1], " .= ", X[:,i] .- X[:,i-1], "\n")
        D_2[:,i] .= X[:,i] .- X[:,i-1]
    end

    return D_2
end

function d3(X)
    X_shape = size(X)
    D_3 = Array(zeros(size(X)))

    for i in 2:X_shape[1]
        for j in 2:X_shape[2]
        print(X[i,j], " - ", X[i-1,j-1], " = ", X[i,j] - X[i-1,j-1], "\n")
        D_3[i,j] = X[i,j] - X[i-1,j-1]
        end
    end

    return D_3
end

function d4(X)
    X_shape = size(X)
    D_4 = Array(zeros(size(X)))

    for i in 2:X_shape[1]
        for j in 2:X_shape[2]-1
        print(X[i,j], " - ", X[i-1,j+1], " = ", X[i,j]-X[i-1,j+1], "\n")
        D_4[i,j] = X[i,j] - X[i-1,j+1]
        end
    end

    return D_4
end

X = Array(reshape(1:24, 4, 6))
display("X:")
display(X)

D = d4(X)

display("D:")
display(D)

"X:"

4×6 Array{Int64,2}:
 1  5   9  13  17  21
 2  6  10  14  18  22
 3  7  11  15  19  23
 4  8  12  16  20  24

"D:"

4×6 Array{Float64,2}:
 0.0   0.0   0.0   0.0   0.0  0.0
 0.0  -3.0  -3.0  -3.0  -3.0  0.0
 0.0  -3.0  -3.0  -3.0  -3.0  0.0
 0.0  -3.0  -3.0  -3.0  -3.0  0.0

6 - 9 = -3
10 - 13 = -3
14 - 17 = -3
18 - 21 = -3
7 - 10 = -3
11 - 14 = -3
15 - 18 = -3
19 - 22 = -3
8 - 11 = -3
12 - 15 = -3
16 - 19 = -3
20 - 23 = -3


In [ ]:
using Random
Random.seed!(0);

Ω = sort(sample(collect(1:256*256), round(Int64, 0.3 * 256 * 256), replace=false));
function P_Ω(x)

end